In [1]:
#require "base";;
#require "core_bench";;
open Base;;
open Core_bench;;

let rec sum_match = function
    | [] -> 0
    | hd :: tl -> hd + sum_match tl;;

let rec sum_ifelse lst =
    if List.is_empty lst then 0
    else List.hd_exn lst + sum_ifelse (List.tl_exn lst);;

let nums = List.range 1 100 in
let bench1 = Bench.Test.create ~name:"sum_match" (fun () -> sum_match nums) and
bench2 = Bench.Test.create ~name:"sum_ifelse" (fun () -> sum_ifelse nums) in
[bench1; bench2] |> Bench.bench;;

/Users/jazz/.opam/4.12.0/lib/base/base_internalhash_types: added to search path
/Users/jazz/.opam/4.12.0/lib/base/base_internalhash_types/base_internalhash_types.cma: loaded
/Users/jazz/.opam/4.12.0/lib/base/caml: added to search path
/Users/jazz/.opam/4.12.0/lib/base/caml/caml.cma: loaded
/Users/jazz/.opam/4.12.0/lib/base/shadow_stdlib: added to search path
/Users/jazz/.opam/4.12.0/lib/base/shadow_stdlib/shadow_stdlib.cma: loaded
/Users/jazz/.opam/4.12.0/lib/sexplib0: added to search path
/Users/jazz/.opam/4.12.0/lib/sexplib0/sexplib0.cma: loaded
/Users/jazz/.opam/4.12.0/lib/base: added to search path
/Users/jazz/.opam/4.12.0/lib/base/base.cma: loaded
/Users/jazz/.opam/4.12.0/lib/fieldslib: added to search path
/Users/jazz/.opam/4.12.0/lib/fieldslib/fieldslib.cma: loaded
/Users/jazz/.opam/4.12.0/lib/ppx_compare/runtime-lib: added to search path
/Users/jazz/.opam/4.12.0/lib/ppx_compare/runtime-lib/ppx_compare_lib.cma: loaded
/Users/jazz/.opam/4.12.0/lib/ppx_enumerate/runtime-lib: added

val sum_match : Base.Int.t list -> Base.Int.t = <fun>


val sum_ifelse : Base.Int.t Base.List.t -> Base.Int.t = <fun>


Estimated testing time 20s (2 benchmarks x 10s). Change using '-quota'.


- : unit = ()


In [ ]:
#require "base";;
open Base;;
(** looping forever

let rec drop to_drop lst =
    match lst with
    | [] -> []
    | hd :: tl -> 
        if hd = to_drop then drop to_drop tl
        else drop to_drop (hd :: tl);;
*)

let rec drop to_drop lst =
    match lst with
    | [] -> []
    | hd :: tl -> 
        let new_tl = drop to_drop tl in
        if hd = to_drop then new_tl else hd :: new_tl;;
drop 10  [10;10;2;2;10];;
drop 10  ([10;10;2;2;10] |> List.sort ~compare:Int.compare);;

In [ ]:
#require "base";;
open Base;;

let header = ["language";"architect";"first release"];;
let rows = [ ["Lisp" ;"John McCarthy" ;"1958"] ;
       ["C"    ;"Dennis Ritchie";"1969"] ;
       ["ML"   ;"Robin Milner"  ;"1973"] ;
       ["OCaml";"Xavier Leroy"  ;"1996"] ;
];;

let max_widths header rows =
    let len l = l |> List.map ~f:String.length in
    rows 
    |> List.fold 
        ~init:(len header)
        ~f:(fun accum row -> List.map2_exn accum (len row) ~f:Int.max);;

max_widths header rows;;

let render_separator widths =
    let ln = 
        widths 
            |> List.map ~f:(fun x -> (String.make x '-')) 
            |> String.concat ~sep:"-+-" in
        "|-" ^ ln ^ "-|" ;;
    
max_widths header rows |> render_separator ;;

let render_row row widths =
    let ln =
        List.map2_exn row widths ~f:(fun r w -> r ^ String.make (w - String.length r) ' ')
        |> String.concat ~sep:" | " in
    "| " ^ ln ^ " |" ;;

render_row ["C"    ;"Dennis Ritchie";"1969"] (max_widths header rows);;

let render_table header rows =
    let widths = max_widths header rows in
        render_row header widths ::
        render_separator widths ::
        (rows |> List.map ~f:(fun r -> render_row r widths)) ;;
        (** |> String.concat ~sep:"\n" *)
    
    
render_table header rows;;

In [ ]:
let filenames = ["foo.c"; "foo.ml"; "bar.ml"; "bar.mli"];;
let extensions filenames =
    filenames
     |> List.filter_map ~f:(fun f -> 
            match String.rsplit2 ~on:'.' f with
            | None | Some(_, "") -> None
            | Some(_, ext) -> Some ext) 
    |> List.dedup_and_sort ~compare:String.compare;;
            
extensions filenames;;

let ocaml_sources filenames =
    filenames
     |> List.partition_tf ~f:(fun f ->
         match String.rsplit2 f ~on:'.' with
         | Some(_,("ml" | "mli")) -> true
         | _ -> false) ;;
        
ocaml_sources filenames;

In [ ]:
open Base;;
let rec ls_rec dir =
    match Core.Sys.is_file dir with
    | `Yes -> [dir]
    | _ -> dir
            |> Core.Sys.ls_dir
            |> List.concat_map ~f:(fun f -> ls_rec (Core.Filename.concat dir f)) ;;


In [ ]:
let rec ls_rec path =
    match Sys_unix.is_file path with
    | `Yes -> [path]
    | _ -> 
        match Sys_unix.is_directory path with
        | `Yes -> Sys_unix.ls_dir path |> List.concat_map ~f:(fun f -> ls_rec (Core.Filename.concat path f))
        | _ -> [];;

In [4]:
#require "base";;
open Base;;

let length l =
    let rec loop l size = 
        match l with
        | [] -> size
        | _ :: tl -> loop tl (size + 1) in
    loop l 0;;

let lst = List.range 0 1000000 in
lst |> length

val length : 'a list -> Base.Int.t = <fun>


- : Base.Int.t = 1000000


In [28]:
#require "base";;
#require "core_bench";;
open Base;;
open Base.Poly;;
open Core_bench;;

let rec remove_sequential_duplicates = function
    | [] | [_] as l -> l
    | f :: (s :: _ as tl) when f = s -> remove_sequential_duplicates tl
    | f :: tl -> f :: remove_sequential_duplicates tl;;
    
remove_sequential_duplicates [1;2;2;2;1;2;2;3;3;4;5];;

(** worst version
    let remove_sequential_duplicates_tc l =
    let rec loop l r =
        match l with
        | [] -> r
        | [hd] -> hd :: r
        | f :: (s :: _ as tl) when f = s -> loop tl r
        | f :: tl -> loop tl (f :: r) in
    loop l [] ;;
    remove_sequential_duplicates_tc (List.init 100000000 (fun _ -> 1));;
    
    let nums = List.init 10000 (fun _ -> 1) in
    let bench1 = Bench.Test.create ~name:"remove_sequential_duplicates" (fun () -> remove_sequential_duplicates nums) and
    bench2 = Bench.Test.create ~name:"remove_sequential_duplicates_tc" (fun () -> remove_sequential_duplicates_tc nums) in
    [bench1; bench2] |> Bench.bench;;
*)






val remove_sequential_duplicates : 'a list -> 'a list = <fun>


- : int list = [1; 2; 1; 2; 3; 4; 5]
